<a href="https://colab.research.google.com/github/aimerou/deep-learning/blob/main/notebooks/sklearn_hyperopt_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hyperopt

In [2]:
from sklearn import datasets, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll import scope

In [3]:
# import some data to play with
iris = datasets.load_iris()
# Take the first two features. We could avoid this by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
def get_clf(params):
  if params['type'] == 'svm':
    clf = svm.SVC(kernel=params['kernel'], C=params['C'])
  elif params['type'] =='random_forest':
    clf = RandomForestClassifier(n_estimators=params['n_estimators'],criterion=params['criterion'], max_depth=params['max_depth'])
  return clf

def objective(params):
  clf = get_clf(params)
  clf.fit(X_train, y_train)
  score = -1 * clf.score(X_test, y_test)
  return score

In [6]:
space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.lognormal('svm_C', 0, 1),
        'kernel': hp.choice('svm_kernel', ['linear', 'rbf'])
    },
    {
        'type': 'random_forest',
        'n_estimators': scope.int(hp.uniform('rf_n_estimators', 10, 100)),
        'criterion': hp.choice('rf_criterion', ['gini', 'entropy']),
        'max_depth': hp.uniform('rf_max_depth_int', 0, 5)
    }
])

In [7]:
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=100)

100%|██████████| 100/100 [00:34<00:00,  2.92it/s, best loss: -0.8666666666666667]


In [8]:
best_params = hyperopt.space_eval(space, best)
clf = get_clf(best_params)
clf.fit(X_train, y_train)
print(f"Best Model : {best_params}, Score : {clf.score(X_test, y_test)}")

Best Model : {'C': 46213.95718173061, 'kernel': 'rbf', 'type': 'svm'}, Score : 0.8666666666666667
